# Ejercicio 3

Se proporciona una BBDD con la descripción de los siniestros de los clientes, tipificados por causa.

El objetivo del ejercicio es usar el modelo de IA que consideres más apropiado y sobre el, hacer un fine tuning para
conseguir que tipifique las causas a partir de la descripción.

Cargamos las librerías

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from scipy.stats import norm
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
import warnings
warnings.simplefilter(action='ignore')
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
import re

2025-02-03 14:03:22.326227: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-03 14:03:22.326300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-03 14:03:22.358885: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-03 14:03:22.432677: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-03 14:03:23.360995: W tensorflow/compiler/tf2

leemos el df

In [2]:
df_siniestros = pd.read_csv('/home/dell/Descargas/Prueba/BBDD_CAUSAS_SINIESTROS_TRAIN.csv', delimiter=';', encoding='latin1')

In [3]:
df_siniestros.head()

,ID_SINIESTRO,DE_TEXTO,DE_TFJ7_CASTELLANO
0,375394764,ROTURA DE GRIFO DE FREGADERO COCINA ESTA OCAS...,Mantenimiento
1,375397056,A consecuenc a del viento se ha despegado del ...,Fenómenos atmosféricos
2,375397700,Rotura de loza sanitaria de tanque de cisterna...,"Cristales, mármol y loza sanitaria"
3,375401437,ASEGURADO INDICA FUGA DE AGUA EN SUELO DE BAñO...,Daños por agua
4,375402120,LLAMA ASEGURADO PARA INDICAR QUE SALE AGUA POR...,Daños por agua


Agrupamos tipologías para reeducir el número de clases

In [4]:
correcciones = {
    'Actos vandálicos y otros': 'vandalismo, robo u otros',  
    'Cristales, mármol y loza sanitaria': 'daños materiales', 
    'Daños Materiales': 'daños materiales', 
    'EXPLOTACIÓN DAÑOS PERSONALES Y MATERIALES': 'daños materiales', 
    'Daños eléctricos': 'incendios, explosiones o daños electricos',
    'Hurto': 'vandalismo, robo u otros',
    'IMPACTO MAQUINARIA': 'daños materiales',
    'Mantenimiento': 'responsabilidad civil o mantenimiento',
    'Incendio': 'incendios, explosiones o daños electricos',
    'OTRAS CAUSAS R.C. ACTIVIDAD COM.': 'responsabilidad civil o mantenimiento',
    'OTRAS CAUSAS R.C. PATRONAL COM.': 'responsabilidad civil o mantenimiento',
    'Otras causas': 'vandalismo, robo u otros',  
    'Otras contingencias': 'vandalismo, robo u otros',  
    'Responsabilidad civil': 'responsabilidad civil o mantenimiento',  
    'Robo': 'vandalismo, robo u otros',  
    'SALPICADURAS MATERIAL DE OBRA O DE MATERIAL DE TRA': 'daños materiales',  
    'otros': 'vandalismo, robo u otros',  
}

df_siniestros['causa'] = df_siniestros['DE_TFJ7_CASTELLANO'].replace(correcciones)

In [5]:
num_classes = df_siniestros["causa"].nunique()  # Número de causas únicas

In [6]:
unique_labels = df_siniestros["causa"].unique()  # Lista de causas únicas

id2label = {i: label for i, label in enumerate(unique_labels)}
label2id = {label: i for i, label in id2label.items()}

En este caso, utilizamos el modelo BETO (variante del modelo BERT) que ha sido entrenado previamente sobre un corpus en español y tendrá mejor rendimiento sobre nuestro dataset

In [7]:
model_name = "dccuchile/bert-base-spanish-wwm-uncased"  # BETO
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_classes,  
    id2label=id2label,       
    label2id=label2id        
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Normalizacion, no hace falta quitar stopwords el modelo puede tratarlas correctamente
def normalizar_texto(texto):
    # Eliminar URLs, menciones y hashtags
    texto = re.sub(r"http\S+|www\S+|https\S+", "", texto, flags=re.MULTILINE)
    texto = re.sub(r"@\w+", "", texto)
    texto = re.sub(r"#\w+", "", texto)
    
    # Convertir a minúsculas 
    texto = texto.lower()
    
    # Eliminar espacios en blanco adicionales
    texto = texto.strip()
    
    return texto

In [9]:
df_siniestros["descripcion_normalizada"] = df_siniestros["DE_TEXTO"].apply(normalizar_texto)
df_siniestros[['descripcion_normalizada', 'causa']].head()

,descripcion_normalizada,causa
0,rotura de grifo de fregadero cocina esta ocas...,responsabilidad civil o mantenimiento
1,a consecuenc a del viento se ha despegado del ...,Fenómenos atmosféricos
2,rotura de loza sanitaria de tanque de cisterna...,daños materiales
3,asegurado indica fuga de agua en suelo de baño...,Daños por agua
4,llama asegurado para indicar que sale agua por...,Daños por agua


In [10]:
df = df_siniestros[['descripcion_normalizada', 'causa']].copy()

In [11]:
# Tokenizacion
inputs = tokenizer(
    list(df['descripcion_normalizada']),          
    padding="max_length",  
    truncation=True,        
    max_length=128,         
    return_tensors="pt"
)

In [12]:
# Mapeamos las etiquetas
le = LabelEncoder()
labels = le.fit_transform(df["causa"])

Realizamos el fine-tuning sobre nuestro conjunto

In [13]:
#fine-tuning

# Convertir datos a formato de Dataset
dataset = Dataset.from_dict({
    "input_ids": inputs["input_ids"],
    "attention_mask": inputs["attention_mask"],
    "labels": labels
}).train_test_split(test_size=0.2)

# Hiperparámetros BETO
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,                  
    per_device_train_batch_size=16,       
    learning_rate=2e-5,                   
    evaluation_strategy="epoch",          
    weight_decay=0.01,                    
    logging_dir="./logs",
)

# Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.608643
2,No log,1.485868
3,No log,1.397871
4,No log,1.356969


TrainOutput(global_step=80, training_loss=1.2719301223754882, metrics={'train_runtime': 331.1208, 'train_samples_per_second': 3.793, 'train_steps_per_second': 0.242, 'total_flos': 82619838517248.0, 'train_loss': 1.2719301223754882, 'epoch': 4.0})

Obtenemos las métricas del modelo

In [14]:
# Predicción en el conjunto de prueba
predictions = trainer.predict(dataset["test"])
preds = np.argmax(predictions.predictions, axis=-1)

# Reporte de clasificación detallado
print(classification_report(
    dataset["test"]["labels"],
    preds,
    target_names=le.classes_  
))

                                           precision    recall  f1-score   support

                           Daños por agua       0.48      0.88      0.62        26
                   Fenómenos atmosféricos       0.00      0.00      0.00         5
                         daños materiales       0.48      0.69      0.56        16
incendios, explosiones o daños electricos       0.00      0.00      0.00         2
    responsabilidad civil o mantenimiento       0.00      0.00      0.00        11
                 vandalismo, robo u otros       0.75      0.32      0.44        19

                                 accuracy                           0.51        79
                                macro avg       0.28      0.31      0.27        79
                             weighted avg       0.43      0.51      0.43        79



Observamos una media ponderada de la métrica f1-score de 0.43, teniendo en cuenta el balanceo existente entre las clases y el reducido número de datos, es un rendimiento aceptable del modelo